## Notebook with feature engineering process

### Curated Features list (in addition to columns)

#### 1 Features based on Sessions actions:  
1. [x] Create features event_i with according to:  
    * event_i means that it's action_info event of order i  
    * take its first order in session, i.e. if events are show_nan_nan, show_view_p3 then values for show_view_p3 is 2  
    * normalize by deviding by total number of events in user's session
2. [x] COUNT for each action_type
3. [x] MEAN, MAX and other descriptive statistics of secs_elapsed deltas

#### 2 Aggregated on Sessions:  
1. [x] COUNT DISTINCT of device_type
2. [ ] % time spent on each action type
3. [ ] count sessions per each device, MODE of Device type  
4. [ ] given that timestamp_first_active is the start of the session, analyze hour (0-23) of activity

#### 3 Transformed from users:
1. [x] Hour of first activity - users['hour_factive'] = users.timestamp_first_active.dt.hour
2. [x] date of week of account_created

**TODO**: use age_gender_bktd and countries data for features generation

In [1]:
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import numpy as np
from scipy import stats
from collections import Counter
from scipy import stats
import itertools

pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
tqdm.pandas()
# %load_ext autotime

### 0. Loading Data

In [2]:
users = pd.read_parquet('../data/processed/users.parquet')
users.shape

(275547, 16)

time: 1.1 s (started: 2021-09-07 12:03:39 +00:00)


In [3]:
sessions = pd.read_parquet('../data/processed/sessions.parquet')
sessions.shape

(10533241, 7)

time: 4.04 s (started: 2021-09-07 12:03:40 +00:00)


In [4]:
users.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,train_flag
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,1
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,MALE,38.00,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,1
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,FEMALE,56.00,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,1
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,FEMALE,42.00,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,1
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,-unknown-,41.00,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,1


time: 43.6 ms (started: 2021-09-07 12:03:44 +00:00)


In [5]:
sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed,action_info
0,d1mm9tcy42,lookup,None,None,Windows Desktop,319.00,lookup_nan_nan
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.00,search_results_click_view_search_results
2,d1mm9tcy42,lookup,None,None,Windows Desktop,301.00,lookup_nan_nan
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.00,search_results_click_view_search_results
4,d1mm9tcy42,lookup,None,None,Windows Desktop,435.00,lookup_nan_nan


time: 26.4 ms (started: 2021-09-07 12:03:44 +00:00)


### 2. Getting features based on Sessions

In [6]:
sessions.secs_elapsed.fillna(-1, inplace=True)
sessions.sort_values(['user_id', 'secs_elapsed'], inplace=True)
sessions.reset_index(drop=True, inplace=True)
sessions.shape

(10533241, 7)

time: 11.3 s (started: 2021-09-07 12:03:44 +00:00)


In [7]:
sessions.head(10)

,user_id,action,action_type,action_detail,device_type,secs_elapsed,action_info
0,00023iyk9l,callback,partner_callback,oauth_response,Mac Desktop,-1.00,callback_partner_callback_oauth_response
1,00023iyk9l,pending,booking_request,pending,Mac Desktop,0.00,pending_booking_request_pending
2,00023iyk9l,personalize,data,wishlist_content_update,Mac Desktop,6.00,personalize_data_wishlist_content_update
3,00023iyk9l,show,None,None,Mac Desktop,45.00,show_nan_nan
4,00023iyk9l,similar_listings,data,similar_listings,Mac Desktop,81.00,similar_listings_data_similar_listings
5,00023iyk9l,similar_listings,data,similar_listings,Mac Desktop,94.00,similar_listings_data_similar_listings
6,00023iyk9l,show,None,None,Mac Desktop,112.00,show_nan_nan
7,00023iyk9l,similar_listings,data,similar_listings,Mac Desktop,155.00,similar_listings_data_similar_listings
8,00023iyk9l,index,view,view_search_results,Mac Desktop,163.00,index_view_view_search_results
9,00023iyk9l,show,view,p3,Mac Desktop,182.00,show_view_p3


time: 23.4 ms (started: 2021-09-07 12:03:56 +00:00)


### 2.1 Generating features based on action_info Events vs its Order in the session stream with Normalization

Sessions actions:  
Create features event_i with according to:  
    * event_i means that it's action_info event of order i  
    * take its first order in session, i.e. if events are show_nan_nan, show_view_p3 then values for show_view_p3 is 2  
    * normalize by deviding by total number of events in user's session  

### Small change to previous approaches, taking only those action_info, which are present in both train and test datasets, without dropping the rest of action_info

In [8]:
# actions_info = list(sessions.action_info.unique())
# len(actions_info)
actions_info = set(sessions[sessions.user_id.isin(users[users.train_flag == 1].id)].action_info.unique())
actions_info = actions_info.intersection(set(sessions[sessions.user_id.isin(users[users.train_flag == 0].id)].action_info.unique()))
actions_info = list(actions_info)
len(actions_info)

336

time: 4.59 s (started: 2021-09-07 12:03:56 +00:00)


In [9]:
# actions_info = list(sessions.action_info.unique())
# len(actions_info)
action_details = set(sessions[sessions.user_id.isin(users[users.train_flag == 1].id)].action_detail.unique())
action_details = action_details.intersection(set(sessions[sessions.user_id.isin(users[users.train_flag == 0].id)].action_detail.unique()))
action_details = list(action_details)
len(action_details)

127

time: 3.78 s (started: 2021-09-07 12:04:00 +00:00)


In [10]:
sessions.action_detail.nunique()

155

time: 951 ms (started: 2021-09-07 12:04:04 +00:00)


In [11]:
tmp = sessions[['user_id', 'action_info']].groupby('user_id', as_index=False).agg(list)
tmp.shape

(135483, 2)

time: 5.36 s (started: 2021-09-07 12:04:05 +00:00)


In [12]:
tmp['size'] = tmp.action_info.apply(lambda x: len(x))

time: 111 ms (started: 2021-09-07 12:04:10 +00:00)


In [13]:
tmp.head()

,user_id,action_info,size
0,00023iyk9l,"[callback_partner_callback_oauth_response, pen...",40
1,0010k6l0om,"[callback_partner_callback_oauth_response, sea...",63
2,001wyh0pz8,"[create_submit_signup, search_click_view_searc...",90
3,0028jgx1x1,"[create_submit_create_user, show_view_user_pro...",31
4,002qnbzfs5,"[campaigns_nan_nan, click_click_book_it, show_...",789


time: 17.8 ms (started: 2021-09-07 12:04:11 +00:00)


In [14]:
tmp.columns = ['user_id', 'action_info', 'seassion_length']

time: 1.64 ms (started: 2021-09-07 12:04:11 +00:00)


In [15]:
def find_action_info_pos(ai, ais):
    try:
        return ais.index(ai) + 1
    except ValueError:
        return None

time: 828 µs (started: 2021-09-07 12:04:11 +00:00)


In [16]:
res = []
for ai in tqdm(actions_info):
    res.append(pd.DataFrame({f'ai_{ai}': tmp.action_info.apply(lambda x: find_action_info_pos(ai, x))}))
tmp = pd.concat([tmp] + res, axis=1)
tmp.shape

  0%|          | 0/336 [00:00<?, ?it/s]

(135483, 339)

time: 2min 12s (started: 2021-09-07 12:04:11 +00:00)


In [17]:
lst = [1, 2, 1, 2, 3, 4, 1, 3, 4, 2]

time: 914 µs (started: 2021-09-07 12:06:23 +00:00)


In [18]:
lst.index(2, 1)

1

time: 10.9 ms (started: 2021-09-07 12:06:23 +00:00)


### 2.1.0 Adding sequencialization of action_info events features

In [19]:
top20_ai = pd.read_csv('../data/processed/top20_ai_features.csv')
top20_ai.shape

(20, 3)

time: 63.5 ms (started: 2021-09-07 12:06:23 +00:00)


In [20]:
top20_ai.head(2)

,feature_name,weight,w_sum
0,ai_pending_booking_request_pending,0.10,0.45
1,ai_nan_message_post_message_post,0.01,0.81


time: 48.6 ms (started: 2021-09-07 12:06:24 +00:00)


In [21]:
top20_ai = top20_ai.feature_name.tolist()
len(top20_ai)

20

time: 7.69 ms (started: 2021-09-07 12:06:24 +00:00)


In [22]:
top20_ai = list(itertools.permutations(top20_ai,2))
len(top20_ai), top20_ai[0]

(380,
 ('ai_pending_booking_request_pending', 'ai_nan_message_post_message_post'))

time: 42.7 ms (started: 2021-09-07 12:06:24 +00:00)


In [23]:
def count_times_a_before_b(a, b, lst):
    pos_a = lst.index(a) if a in lst else None
    pos_b = lst.index(b, pos_a) if b in lst and pos_a is not None else None
    count = 0
    while pos_a is not None and pos_b is not None:
        count += 1
        pos_a = lst.index(a, pos_a + 1) if a in lst[pos_a + 1:] else None
        pos_b = lst.index(b, pos_a) if b in lst[pos_a:] and pos_a is not None else None
    return count

time: 6.25 ms (started: 2021-09-07 12:06:24 +00:00)


In [24]:
res = []
for (a, b) in tqdm(top20_ai):
    res.append(pd.DataFrame({f'ai_{a}_before_{b}': tmp.action_info.apply(lambda x: count_times_a_before_b(a, b, x))}))
tmp = pd.concat([tmp] + res, axis=1)
tmp.shape

  0%|          | 0/380 [00:00<?, ?it/s]

(135483, 719)

time: 3min 16s (started: 2021-09-07 12:06:24 +00:00)


In [25]:
tmp.head()

,user_id,action_info,seassion_length,ai_forgot_password_click_forgot_password,ai_create_submit_create_user,ai_confirm_email_click_confirm_email_link,ai_domains_-unknown-_-unknown-,ai_set_user_submit_create_listing,ai_phone_verification_nan_nan,ai_references_view_profile_references,ai_15_message_post_message_post,ai_create_paypal_-unknown-_-unknown-,ai_create_submit_signup,ai_click_click_request_to_book,ai_populate_from_facebook_-unknown-_-unknown-,ai_patch_-unknown-_-unknown-,ai_ajax_google_translate_description_-unknown-_-unknown-,ai_11_message_post_message_post,ai_terms_view_terms_and_privacy,ai_show_view_view_listing,ai_founders_-unknown-_-unknown-,ai_transaction_history_paginated_-unknown-_-unknown-,ai_payment_methods_-unknown-_-unknown-,ai_mobile_landing_page_-unknown-_-unknown-,ai_apply_coupon_click_success_click_apply_coupon_click_success,ai_envoy_form_-unknown-_-unknown-,ai_qt_with_data_lookup_message_thread,ai_header_userpic_data_header_userpic,ai_authenticate_submit_login,ai_track_activity_nan_nan,ai_delete_-unknown-_-unknown-,ai_payoneer_signup_complete_-unknown-_-unknown-,ai_update_submit_update_listing_description,ai_campaigns_nan_nan,ai_listings_-unknown-_-unknown-,ai_show_view_alteration_request,ai_cancellation_policy_click_click_cancellation_policy_click,ai_create_multiple_-unknown-_-unknown-,ai_toggle_archived_thread_click_toggle_archived_thread,ai_trust_-unknown-_-unknown-,ai_views_-unknown-_-unknown-,ai_index_data_reservations,ai_ajax_special_offer_dates_available_click_special_offer_field,ai_reputation_-unknown-_-unknown-,ai_glob_-unknown-_-unknown-,ai_badge_-unknown-_-unknown-,ai_clear_reservation_-unknown-_-unknown-,ai_special_offer_message_post_message_post,ai_ajax_lwlb_contact_click_contact_host,ai_remove_dashboard_alert_-unknown-_-unknown-,ai_impressions_view_p4,ai_complete_-unknown-_-unknown-,ai_southern-europe_-unknown-_-unknown-,ai_update_submit_update_user_profile,ai_add_note_submit_wishlist_note,ai_recent_reservations_-unknown-_-unknown-,ai_index_data_user_tax_forms,ai_recommendations_data_listing_recommendations,ai_show_view_wishlist,ai_media_resources_-unknown-_-unknown-,ai_add_guests_-unknown-_-unknown-,ai_invalid_action_-unknown-_-unknown-,ai_click_click_contact_host,ai_kba_-unknown-_-unknown-,ai_qt_reply_v2_-unknown-_-unknown-,ai_home_safety_landing_-unknown-_-unknown-,ai_index_view_view_search_results,ai_edit_-unknown-_-unknown-,ai_show_view_p3,ai_approve_-unknown-_-unknown-,ai_complete_redirect_-unknown-_-unknown-,ai_phone_verification_number_sucessfully_submitted_-unknown-_-unknown-,ai_phone_verification_phone_number_removed_-unknown-_-unknown-,ai_ajax_google_translate_-unknown-_-unknown-,ai_localized_-unknown-_-unknown-,ai_index_view_message_thread,ai_update_submit_update_user,ai_nan_message_post_message_post,ai_terms_and_conditions_-unknown-_-unknown-,ai_change_default_payout_-unknown-_-unknown-,ai_check_nan_nan,ai_has_profile_pic_-unknown-_-unknown-,ai_update_nan_nan,ai_connect_submit_oauth_login,ai_preapproval_message_post_message_post,ai_delete_submit_delete_listing,ai_dashboard_view_dashboard,ai_uptodate_nan_nan,ai_create_ach_-unknown-_-unknown-,ai_track_page_view_nan_nan,ai_confirm_email_click_confirm_email,ai_travel_plans_previous_view_previous_trips,ai_coupon_field_focus_click_coupon_field_focus,ai_languages_multiselect_-unknown-_-unknown-,ai_guest_billing_receipt_-unknown-_-unknown-,ai_decision_tree_-unknown-_-unknown-,ai_recommend_-unknown-_-unknown-,ai_pending_-unknown-_-unknown-,ai_show_view_p1,ai_new_session_-unknown-_-unknown-,ai_click_click_book_it,ai_locations_-unknown-_-unknown-,ai_department_-unknown-_-unknown-,ai_create_-unknown-_-unknown-,ai_search_results_click_view_search_results,ai_authenticate_view_login_page,ai_my_listings_view_your_reservations,ai_other_hosting_reviews_-unknown-_-unknown-,ai_agree_terms_check_-unknown-_-unknown-,ai_qt_reply_v2_submit_send_message,ai_overview_-unknown-_-unknown-,ai_calendar_tab_inner2_-unknown-_-unknown-,ai_countries_-unknown-_-un

time: 492 ms (started: 2021-09-07 12:09:40 +00:00)


### 2.1.b Adding action_info counts features

In [26]:
def get_action_info_count(ai, ais):
    return ais.count(ai)

time: 1.04 ms (started: 2021-09-07 12:09:41 +00:00)


In [27]:
res = []
for ai in tqdm(actions_info):
    res.append(pd.DataFrame({f'count_{ai}': tmp.action_info.apply(lambda x: get_action_info_count(ai, x))}))
tmp = pd.concat([tmp] + res, axis=1)
tmp.shape

  0%|          | 0/336 [00:00<?, ?it/s]

(135483, 1055)

time: 2min 11s (started: 2021-09-07 12:09:41 +00:00)


In [28]:
tmp.head()

,user_id,action_info,seassion_length,ai_forgot_password_click_forgot_password,ai_create_submit_create_user,ai_confirm_email_click_confirm_email_link,ai_domains_-unknown-_-unknown-,ai_set_user_submit_create_listing,ai_phone_verification_nan_nan,ai_references_view_profile_references,ai_15_message_post_message_post,ai_create_paypal_-unknown-_-unknown-,ai_create_submit_signup,ai_click_click_request_to_book,ai_populate_from_facebook_-unknown-_-unknown-,ai_patch_-unknown-_-unknown-,ai_ajax_google_translate_description_-unknown-_-unknown-,ai_11_message_post_message_post,ai_terms_view_terms_and_privacy,ai_show_view_view_listing,ai_founders_-unknown-_-unknown-,ai_transaction_history_paginated_-unknown-_-unknown-,ai_payment_methods_-unknown-_-unknown-,ai_mobile_landing_page_-unknown-_-unknown-,ai_apply_coupon_click_success_click_apply_coupon_click_success,ai_envoy_form_-unknown-_-unknown-,ai_qt_with_data_lookup_message_thread,ai_header_userpic_data_header_userpic,ai_authenticate_submit_login,ai_track_activity_nan_nan,ai_delete_-unknown-_-unknown-,ai_payoneer_signup_complete_-unknown-_-unknown-,ai_update_submit_update_listing_description,ai_campaigns_nan_nan,ai_listings_-unknown-_-unknown-,ai_show_view_alteration_request,ai_cancellation_policy_click_click_cancellation_policy_click,ai_create_multiple_-unknown-_-unknown-,ai_toggle_archived_thread_click_toggle_archived_thread,ai_trust_-unknown-_-unknown-,ai_views_-unknown-_-unknown-,ai_index_data_reservations,ai_ajax_special_offer_dates_available_click_special_offer_field,ai_reputation_-unknown-_-unknown-,ai_glob_-unknown-_-unknown-,ai_badge_-unknown-_-unknown-,ai_clear_reservation_-unknown-_-unknown-,ai_special_offer_message_post_message_post,ai_ajax_lwlb_contact_click_contact_host,ai_remove_dashboard_alert_-unknown-_-unknown-,ai_impressions_view_p4,ai_complete_-unknown-_-unknown-,ai_southern-europe_-unknown-_-unknown-,ai_update_submit_update_user_profile,ai_add_note_submit_wishlist_note,ai_recent_reservations_-unknown-_-unknown-,ai_index_data_user_tax_forms,ai_recommendations_data_listing_recommendations,ai_show_view_wishlist,ai_media_resources_-unknown-_-unknown-,ai_add_guests_-unknown-_-unknown-,ai_invalid_action_-unknown-_-unknown-,ai_click_click_contact_host,ai_kba_-unknown-_-unknown-,ai_qt_reply_v2_-unknown-_-unknown-,ai_home_safety_landing_-unknown-_-unknown-,ai_index_view_view_search_results,ai_edit_-unknown-_-unknown-,ai_show_view_p3,ai_approve_-unknown-_-unknown-,ai_complete_redirect_-unknown-_-unknown-,ai_phone_verification_number_sucessfully_submitted_-unknown-_-unknown-,ai_phone_verification_phone_number_removed_-unknown-_-unknown-,ai_ajax_google_translate_-unknown-_-unknown-,ai_localized_-unknown-_-unknown-,ai_index_view_message_thread,ai_update_submit_update_user,ai_nan_message_post_message_post,ai_terms_and_conditions_-unknown-_-unknown-,ai_change_default_payout_-unknown-_-unknown-,ai_check_nan_nan,ai_has_profile_pic_-unknown-_-unknown-,ai_update_nan_nan,ai_connect_submit_oauth_login,ai_preapproval_message_post_message_post,ai_delete_submit_delete_listing,ai_dashboard_view_dashboard,ai_uptodate_nan_nan,ai_create_ach_-unknown-_-unknown-,ai_track_page_view_nan_nan,ai_confirm_email_click_confirm_email,ai_travel_plans_previous_view_previous_trips,ai_coupon_field_focus_click_coupon_field_focus,ai_languages_multiselect_-unknown-_-unknown-,ai_guest_billing_receipt_-unknown-_-unknown-,ai_decision_tree_-unknown-_-unknown-,ai_recommend_-unknown-_-unknown-,ai_pending_-unknown-_-unknown-,ai_show_view_p1,ai_new_session_-unknown-_-unknown-,ai_click_click_book_it,ai_locations_-unknown-_-unknown-,ai_department_-unknown-_-unknown-,ai_create_-unknown-_-unknown-,ai_search_results_click_view_search_results,ai_authenticate_view_login_page,ai_my_listings_view_your_reservations,ai_other_hosting_reviews_-unknown-_-unknown-,ai_agree_terms_check_-unknown-_-unknown-,ai_qt_reply_v2_submit_send_message,ai_overview_-unknown-_-unknown-,ai_calendar_tab_inner2_-unknown-_-unknown-,ai_countries_-unknown-_-un

time: 807 ms (started: 2021-09-07 12:11:52 +00:00)


In [29]:
tmp.drop('action_info', axis=1, inplace=True)

time: 3.28 s (started: 2021-09-07 12:11:53 +00:00)


#### Checking counts of missing values per each column

In [30]:
# not_missing = pd.DataFrame(tmp.notna().sum()).reset_index()
# not_missing.columns = ['col', 'counts']
# not_missing['ratio'] = not_missing['counts'].apply(lambda x: round(x / len(users), 4))
# not_missing.shape

time: 877 µs (started: 2021-09-07 12:11:56 +00:00)


In [31]:
# not_missing.head()

time: 535 µs (started: 2021-09-07 12:11:56 +00:00)


In [32]:
# threshold = 0.00005
# mask = not_missing.ratio > threshold
# mask.sum()

time: 911 µs (started: 2021-09-07 12:11:56 +00:00)


#### Dropping all columns that are lower than the above threshold
Decided not to do that in this iteration

In [33]:
# keep_columns = not_missing[mask].col.tolist()
# len(keep_columns)

time: 14.5 ms (started: 2021-09-07 12:11:56 +00:00)


In [34]:
# keep_columns[0], keep_columns[-1]

time: 3.92 ms (started: 2021-09-07 12:11:56 +00:00)


### 2.1.c Saving features

In [35]:
# features1 = tmp[keep_columns].copy(deep=True)
features1 = tmp.copy(deep=True)
features1.shape

(135483, 1054)

time: 269 ms (started: 2021-09-07 12:11:56 +00:00)


### 2.1.1 Count of each action_type normalized

In [36]:
col = 'action_type'
col_values = list(sessions[col].unique())
len(col_values)

11

time: 993 ms (started: 2021-09-07 12:11:56 +00:00)


In [37]:
tmp = sessions[['user_id', col]].groupby('user_id', as_index=False).agg(list)
tmp.shape

(135483, 2)

time: 7.19 s (started: 2021-09-07 12:11:57 +00:00)


In [38]:
tmp['size'] = tmp[col].apply(lambda x: len(x))

time: 155 ms (started: 2021-09-07 12:12:05 +00:00)


In [39]:
tmp['counts'] = tmp[col].apply(lambda x: dict(Counter(x)))

time: 1.35 s (started: 2021-09-07 12:12:05 +00:00)


In [40]:
tmp.head()

,user_id,action_type,size,counts
0,00023iyk9l,"[partner_callback, booking_request, data, None...",40,"{'partner_callback': 1, 'booking_request': 1, ..."
1,0010k6l0om,"[partner_callback, click, None, view, None, No...",63,"{'partner_callback': 1, 'click': 16, None: 15,..."
2,001wyh0pz8,"[submit, click, click, -unknown-, -unknown-, c...",90,"{'submit': 3, 'click': 66, '-unknown-': 6, 'vi..."
3,0028jgx1x1,"[submit, view, view, data, view, data, view, d...",31,"{'submit': 1, 'view': 15, 'data': 5, '-unknown..."
4,002qnbzfs5,"[None, click, view, data, click, view, data, v...",789,"{None: 77, 'click': 140, 'view': 216, 'data': ..."


time: 20.1 ms (started: 2021-09-07 12:12:06 +00:00)


In [41]:
tmp = pd.concat([tmp, pd.json_normalize(tmp['counts'])], axis=1)

time: 2.05 s (started: 2021-09-07 12:12:06 +00:00)


In [42]:
tmp.drop(['action_type', 'counts'], axis=1, inplace=True)

time: 45.5 ms (started: 2021-09-07 12:12:08 +00:00)


In [43]:
tmp.head()

,user_id,size,partner_callback,booking_request,data,NaN,view,click,message_post,-unknown-,submit,modify,booking_response
0,00023iyk9l,40,1.00,1.00,9.00,3.00,21.00,4.00,1.00,NaN,NaN,NaN,NaN
1,0010k6l0om,63,1.00,NaN,9.00,15.00,17.00,16.00,NaN,5.00,NaN,NaN,NaN
2,001wyh0pz8,90,NaN,NaN,2.00,5.00,8.00,66.00,NaN,6.00,3.00,NaN,NaN
3,0028jgx1x1,31,NaN,NaN,5.00,NaN,15.00,9.00,NaN,1.00,1.00,NaN,NaN
4,002qnbzfs5,789,NaN,1.00,140.00,77.00,216.00,140.00,16.00,184.00,15.00,NaN,NaN


time: 25.5 ms (started: 2021-09-07 12:12:08 +00:00)


In [44]:
cols = list(tmp)[2:]
cols = [f'at_{e}_counts' for e in cols]

time: 1.15 ms (started: 2021-09-07 12:12:08 +00:00)


In [45]:
tmp.columns = ['user_id', 'size'] + cols

time: 2.1 ms (started: 2021-09-07 12:12:08 +00:00)


In [46]:
# for e in cols:
#     tmp[e] = tmp[e] / tmp['size']

time: 1.02 ms (started: 2021-09-07 12:12:08 +00:00)


In [47]:
tmp.head()

,user_id,size,at_partner_callback_counts,at_booking_request_counts,at_data_counts,at_None_counts,at_view_counts,at_click_counts,at_message_post_counts,at_-unknown-_counts,at_submit_counts,at_modify_counts,at_booking_response_counts
0,00023iyk9l,40,1.00,1.00,9.00,3.00,21.00,4.00,1.00,NaN,NaN,NaN,NaN
1,0010k6l0om,63,1.00,NaN,9.00,15.00,17.00,16.00,NaN,5.00,NaN,NaN,NaN
2,001wyh0pz8,90,NaN,NaN,2.00,5.00,8.00,66.00,NaN,6.00,3.00,NaN,NaN
3,0028jgx1x1,31,NaN,NaN,5.00,NaN,15.00,9.00,NaN,1.00,1.00,NaN,NaN
4,002qnbzfs5,789,NaN,1.00,140.00,77.00,216.00,140.00,16.00,184.00,15.00,NaN,NaN


time: 25.3 ms (started: 2021-09-07 12:12:08 +00:00)


In [48]:
# tmp.drop(['size'], axis=1, inplace=True)

time: 1.14 ms (started: 2021-09-07 12:12:08 +00:00)


In [49]:
tmp.fillna(0, inplace=True)

time: 56.3 ms (started: 2021-09-07 12:12:08 +00:00)


In [50]:
tmp.head()

,user_id,size,at_partner_callback_counts,at_booking_request_counts,at_data_counts,at_None_counts,at_view_counts,at_click_counts,at_message_post_counts,at_-unknown-_counts,at_submit_counts,at_modify_counts,at_booking_response_counts
0,00023iyk9l,40,1.00,1.00,9.00,3.00,21.00,4.00,1.00,0.00,0.00,0.00,0.00
1,0010k6l0om,63,1.00,0.00,9.00,15.00,17.00,16.00,0.00,5.00,0.00,0.00,0.00
2,001wyh0pz8,90,0.00,0.00,2.00,5.00,8.00,66.00,0.00,6.00,3.00,0.00,0.00
3,0028jgx1x1,31,0.00,0.00,5.00,0.00,15.00,9.00,0.00,1.00,1.00,0.00,0.00
4,002qnbzfs5,789,0.00,1.00,140.00,77.00,216.00,140.00,16.00,184.00,15.00,0.00,0.00


time: 23.2 ms (started: 2021-09-07 12:12:08 +00:00)


In [51]:
features1a = tmp.copy(deep=True)
features1a.shape

(135483, 13)

time: 19.9 ms (started: 2021-09-07 12:12:09 +00:00)


### 2.2 Generating features based on seconds elapsed and deltas between info

In [52]:
tmp = sessions[['user_id', 'secs_elapsed']].groupby('user_id', as_index=False).agg(list)
tmp.shape

(135483, 2)

time: 6.88 s (started: 2021-09-07 12:12:09 +00:00)


In [53]:
tmp.head()

,user_id,secs_elapsed
0,00023iyk9l,"[-1.0, 0.0, 6.0, 45.0, 81.0, 94.0, 112.0, 155...."
1,0010k6l0om,"[-1.0, 3.0, 9.0, 22.0, 26.0, 30.0, 34.0, 36.0,..."
2,001wyh0pz8,"[-1.0, 35.0, 80.0, 91.0, 108.0, 118.0, 142.0, ..."
3,0028jgx1x1,"[-1.0, 3.0, 5.0, 20.0, 28.0, 75.0, 86.0, 91.0,..."
4,002qnbzfs5,"[-1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."


time: 19.7 ms (started: 2021-09-07 12:12:15 +00:00)


In [54]:
tmp.secs_elapsed = tmp.secs_elapsed.apply(lambda x: [0] + x[1:])

time: 1.28 s (started: 2021-09-07 12:12:15 +00:00)


In [55]:
tmp['deltas'] = tmp['secs_elapsed'].apply(lambda x: [int(j - i) for i, j in zip(x[:-1], x[1:])])

time: 2.71 s (started: 2021-09-07 12:12:17 +00:00)


In [56]:
tmp.head()

,user_id,secs_elapsed,deltas
0,00023iyk9l,"[0, 0.0, 6.0, 45.0, 81.0, 94.0, 112.0, 155.0, ...","[0, 6, 39, 36, 13, 18, 43, 8, 19, 187, 26, 15,..."
1,0010k6l0om,"[0, 3.0, 9.0, 22.0, 26.0, 30.0, 34.0, 36.0, 39...","[3, 6, 13, 4, 4, 4, 2, 3, 6, 1, 3, 4, 30, 8, 1..."
2,001wyh0pz8,"[0, 35.0, 80.0, 91.0, 108.0, 118.0, 142.0, 201...","[35, 45, 11, 17, 10, 24, 59, 1, 54, 100, 2, 9,..."
3,0028jgx1x1,"[0, 3.0, 5.0, 20.0, 28.0, 75.0, 86.0, 91.0, 97...","[3, 2, 15, 8, 47, 11, 5, 6, 2, 15, 13, 19, 76,..."
4,002qnbzfs5,"[0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


time: 30.2 ms (started: 2021-09-07 12:12:19 +00:00)


In [57]:
tmp.deltas = tmp.deltas.apply(lambda x: np.array(x) if x else None)

time: 2.36 s (started: 2021-09-07 12:12:20 +00:00)


In [58]:
tmp.secs_elapsed = tmp.secs_elapsed.apply(lambda x: np.array(x) if x else None)

time: 2.94 s (started: 2021-09-07 12:12:22 +00:00)


In [59]:
def get_statistics(x):
    if x is None:
        return None, None, None, None, None, None, None
    m = stats.mode(x)
    return x.mean(), x.std(), x.max(), x.min(), np.median(x), m.mode[0], m.count[0]

time: 4.33 ms (started: 2021-09-07 12:12:25 +00:00)


In [60]:
def get_statistics_no_outliers(x):
    if x is None:
        return None, None, None, None, None
    x = np.array(x)
    initial_size = len(x)
    x = x[x  <= x.mean() + x.std()]
    outliers_count = initial_size - len(x)
    m = stats.mode(x)
    return x.mean(), x.std(), x.max(), np.median(x), outliers_count

time: 4.22 ms (started: 2021-09-07 12:12:25 +00:00)


In [61]:
get_statistics(tmp.iloc[0].deltas)

(14542.692307692309, 69958.77759379552, 437348, 0, 64.0, 0, 2)

time: 15.2 ms (started: 2021-09-07 12:12:25 +00:00)


In [62]:
get_statistics_no_outliers(tmp.iloc[0].deltas)

(1165.1351351351352, 2758.241156794633, 11029, 58.0, 2)

time: 18.4 ms (started: 2021-09-07 12:12:25 +00:00)


In [63]:
tmp = pd.concat([tmp, tmp.deltas.progress_apply(lambda x: pd.Series(get_statistics(x)))], axis=1)
tmp.shape

  0%|          | 0/135483 [00:00<?, ?it/s]

(135483, 10)

time: 2min 19s (started: 2021-09-07 12:12:25 +00:00)


In [64]:
column_names = ['user_id', 'secs_elapsed', 'deltas', 'deltas_mean', 'deltas_std', 'deltas_max', 
    'deltas_min', 'deltas_median', 'deltas_mode', 'deltas_mode_count']

time: 1.73 ms (started: 2021-09-07 12:14:44 +00:00)


In [65]:
tmp.columns = column_names

time: 6.39 ms (started: 2021-09-07 12:14:44 +00:00)


In [66]:
tmp.head()

,user_id,secs_elapsed,deltas,deltas_mean,deltas_std,deltas_max,deltas_min,deltas_median,deltas_mode,deltas_mode_count
0,00023iyk9l,"[0.0, 0.0, 6.0, 45.0, 81.0, 94.0, 112.0, 155.0...","[0, 6, 39, 36, 13, 18, 43, 8, 19, 187, 26, 15,...",14542.69,69958.78,437348.00,0.00,64.00,0.00,2.00
1,0010k6l0om,"[0.0, 3.0, 9.0, 22.0, 26.0, 30.0, 34.0, 36.0, ...","[3, 6, 13, 4, 4, 4, 2, 3, 6, 1, 3, 4, 30, 8, 1...",2062.87,6002.06,34874.00,0.00,38.50,2.00,4.00
2,001wyh0pz8,"[0.0, 35.0, 80.0, 91.0, 108.0, 118.0, 142.0, 2...","[35, 45, 11, 17, 10, 24, 59, 1, 54, 100, 2, 9,...",567.96,3206.52,30047.00,0.00,33.00,2.00,4.00
3,0028jgx1x1,"[0.0, 3.0, 5.0, 20.0, 28.0, 75.0, 86.0, 91.0, ...","[3, 2, 15, 8, 47, 11, 5, 6, 2, 15, 13, 19, 76,...",2821.20,7456.50,37388.00,2.00,170.00,2.00,2.00
4,002qnbzfs5,"[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",1799.85,34343.17,946465.00,0.00,3.00,0.00,180.00


time: 89.7 ms (started: 2021-09-07 12:14:44 +00:00)


In [67]:
tmp = pd.concat([tmp, tmp.deltas.progress_apply(lambda x: pd.Series(get_statistics_no_outliers(x)))], axis=1)
tmp.shape

  0%|          | 0/135483 [00:00<?, ?it/s]

(135483, 15)

time: 2min 25s (started: 2021-09-07 12:14:44 +00:00)


In [68]:
tmp.columns = column_names + ['deltas_no_mean', 'deltas_no_std', 'deltas_no_max', 'deltas_no_median', 'deltas_no_num_outliers']

time: 1.1 ms (started: 2021-09-07 12:17:10 +00:00)


In [69]:
tmp.head(2)

,user_id,secs_elapsed,deltas,deltas_mean,deltas_std,deltas_max,deltas_min,deltas_median,deltas_mode,deltas_mode_count,deltas_no_mean,deltas_no_std,deltas_no_max,deltas_no_median,deltas_no_num_outliers
0,00023iyk9l,"[0.0, 0.0, 6.0, 45.0, 81.0, 94.0, 112.0, 155.0...","[0, 6, 39, 36, 13, 18, 43, 8, 19, 187, 26, 15,...",14542.69,69958.78,437348.00,0.00,64.00,0.00,2.00,1165.14,2758.24,11029.00,58.00,2.00
1,0010k6l0om,"[0.0, 3.0, 9.0, 22.0, 26.0, 30.0, 34.0, 36.0, ...","[3, 6, 13, 4, 4, 4, 2, 3, 6, 1, 3, 4, 30, 8, 1...",2062.87,6002.06,34874.00,0.00,38.50,2.00,4.00,692.14,1599.51,8000.00,27.00,4.00


time: 39.7 ms (started: 2021-09-07 12:17:10 +00:00)


### 2.2.1 Adding stats on seconds elapsed

In [70]:
tmp = pd.concat([tmp, tmp.secs_elapsed.progress_apply(lambda x: pd.Series(get_statistics(x)))], axis=1)
tmp.shape

  0%|          | 0/135483 [00:00<?, ?it/s]

(135483, 22)

time: 2min 11s (started: 2021-09-07 12:17:10 +00:00)


In [71]:
tmp.columns = column_names + ['deltas_no_mean', 'deltas_no_std', 'deltas_no_max', 'deltas_no_median', 'deltas_no_num_outliers',
    'secs_elapsed_mean', 'secs_elapsed_std', 'secs_elapsed_max', 'secs_elapsed_min', 'secs_elapsed_median',
    'secs_elapsed_mode', 'secs_elapsed_mode_count'
]

time: 3.11 ms (started: 2021-09-07 12:19:21 +00:00)


In [72]:
tmp.drop(['secs_elapsed', 'deltas'], axis=1, inplace=True)

time: 61.3 ms (started: 2021-09-07 12:19:21 +00:00)


In [73]:
features2 = tmp.copy(deep=True)
features2.shape

(135483, 20)

time: 11 ms (started: 2021-09-07 12:19:21 +00:00)


### 2.3 Generating features based on device type info

In [74]:
tmp = sessions[['user_id', 'device_type']].groupby('user_id', as_index=False).agg(set)
tmp.shape

(135483, 2)

time: 5.62 s (started: 2021-09-07 12:19:21 +00:00)


In [75]:
tmp['size'] = tmp.device_type.apply(lambda x: len(x))

time: 96.5 ms (started: 2021-09-07 12:19:27 +00:00)


In [76]:
tmp.drop('device_type', axis=1, inplace=True)

time: 59.4 ms (started: 2021-09-07 12:19:27 +00:00)


In [77]:
tmp.head()

,user_id,size
0,00023iyk9l,2
1,0010k6l0om,1
2,001wyh0pz8,1
3,0028jgx1x1,2
4,002qnbzfs5,2


time: 17.2 ms (started: 2021-09-07 12:19:27 +00:00)


In [78]:
tmp.columns = ['user_id', 'device_count']

time: 1.37 ms (started: 2021-09-07 12:19:27 +00:00)


In [79]:
tmp.head()

,user_id,device_count
0,00023iyk9l,2
1,0010k6l0om,1
2,001wyh0pz8,1
3,0028jgx1x1,2
4,002qnbzfs5,2


time: 18.3 ms (started: 2021-09-07 12:19:27 +00:00)


In [80]:
features3 = tmp.copy(deep=True)
features3.shape

(135483, 2)

time: 16.1 ms (started: 2021-09-07 12:19:27 +00:00)


### 3.1 Features based on Users table

In [81]:
users[users.train_flag == 1].date_account_created.dt.year.value_counts()

2013    82960
2014    76466
2012    39462
2011    11775
2010     2788
Name: date_account_created, dtype: int64

time: 137 ms (started: 2021-09-07 12:19:27 +00:00)


In [82]:
users[users.train_flag == 0].date_account_created.dt.year.value_counts()

2014    62096
Name: date_account_created, dtype: int64

time: 64.2 ms (started: 2021-09-07 12:19:28 +00:00)


In [83]:
users[users.train_flag == 1].date_account_created.dt.month.value_counts()

6     27422
5     25912
4     21867
3     20325
1     17495
2     16582
9     15395
8     14611
7     13832
10    13476
12    13405
11    13129
Name: date_account_created, dtype: int64

time: 84.4 ms (started: 2021-09-07 12:19:28 +00:00)


In [84]:
users[users.train_flag == 0].date_account_created.dt.month.value_counts()

7    21696
8    21626
9    18774
Name: date_account_created, dtype: int64

time: 78.1 ms (started: 2021-09-07 12:19:28 +00:00)


In [85]:
users['dow_registered'] = users.date_account_created.dt.weekday
users['day_registered'] = users.date_account_created.dt.day
users['month_registered'] = users.date_account_created.dt.month
users['year_registered'] = users.date_account_created.dt.year

time: 154 ms (started: 2021-09-07 12:19:28 +00:00)


In [86]:
users['hr_registered'] = users.timestamp_first_active.dt.hour

time: 37.5 ms (started: 2021-09-07 12:19:28 +00:00)


In [87]:
users.age.max()

2014.0

time: 15.4 ms (started: 2021-09-07 12:19:28 +00:00)


In [88]:
users.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,train_flag,dow_registered,day_registered,month_registered,year_registered,hr_registered
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,1,0,28,6,2010,4
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,MALE,38.00,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,1,2,25,5,2011,17
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,FEMALE,56.00,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,1,1,28,9,2010,23
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,FEMALE,42.00,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,1,0,5,12,2011,6
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,-unknown-,41.00,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,1,1,14,9,2010,6


time: 40.4 ms (started: 2021-09-07 12:19:28 +00:00)


In [89]:
mask = (users.age > 1000) & (users.age < 2000)
users.loc[mask, 'age'] = 2015 - users.loc[mask, 'age']
mask.sum()

71

time: 17.9 ms (started: 2021-09-07 12:19:28 +00:00)


In [90]:
users.loc[(users['age'] > 105) | (users['age'] < 14), 'age'] = -1
users['age'].fillna(-1, inplace=True)

time: 11.4 ms (started: 2021-09-07 12:19:28 +00:00)


In [91]:
bins = [-1, 20, 25, 30, 40, 50, 60, 75, 85, 105]
users['age_group'] = np.digitize(users['age'], bins, right=True)

time: 10.8 ms (started: 2021-09-07 12:19:28 +00:00)


In [92]:
users.sample(5)

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,train_flag,dow_registered,day_registered,month_registered,year_registered,hr_registered,age_group
49123,td650jhx25,2014-09-10,2014-09-10 01:58:25,MALE,30.00,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,None,0,2,10,9,2014,1,3
23023,koqvn6160f,2012-05-03,2012-05-03 01:19:21,FEMALE,29.00,basic,0,en,sem-brand,google,omg,Web,Windows Desktop,Chrome,NDF,1,3,3,5,2012,1,3
30919,cua9kvloie,2014-08-13,2014-08-13 23:24:26,-unknown-,-1.00,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-,None,0,2,13,8,2014,23,0
97754,qyqn42trdh,2013-08-08,2013-08-08 20:15:10,FEMALE,33.00,basic,0,en,direct,direct,untracked,Web,Windows Desktop,Chrome,US,1,3,8,8,2013,20,4
93620,4ttik6i8jx,2013-07-24,2013-07-24 17:01:10,-unknown-,34.00,basic,0,en,sem-brand,google,untracked,Web,iPhone,-unknown-,US,1,2,24,7,2013,17,4


time: 60.8 ms (started: 2021-09-07 12:19:28 +00:00)


In [93]:
users.shape

(275547, 22)

time: 8.13 ms (started: 2021-09-07 12:19:28 +00:00)


### 3.1.1. Dropping redundand columns

In [94]:
users.drop(['date_account_created', 'timestamp_first_active'], axis=1, inplace=True)

time: 46 ms (started: 2021-09-07 12:19:28 +00:00)


In [95]:
users.columns = ['user_id'] + list(users)[1:]

time: 1.89 ms (started: 2021-09-07 12:19:28 +00:00)


In [96]:
users.head()

,user_id,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,train_flag,dow_registered,day_registered,month_registered,year_registered,hr_registered,age_group
0,gxn3p5htnn,-unknown-,-1.00,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,1,0,28,6,2010,4,0
1,820tgsjxq7,MALE,38.00,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,1,2,25,5,2011,17,4
2,4ft3gnwmtx,FEMALE,56.00,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,1,1,28,9,2010,23,6
3,bjjt8pjhuk,FEMALE,42.00,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,1,0,5,12,2011,6,5
4,87mebub9p4,-unknown-,41.00,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,1,1,14,9,2010,6,5


time: 41.7 ms (started: 2021-09-07 12:19:28 +00:00)


In [97]:
users.shape

(275547, 20)

time: 11.5 ms (started: 2021-09-07 12:19:28 +00:00)


#### 4. Assembling all features into one dataset

In [98]:
df = users.merge(features1, on='user_id', how='left')
df.shape

(275547, 1073)

time: 23.6 s (started: 2021-09-07 12:19:28 +00:00)


In [99]:
df = df.merge(features1a, on='user_id', how='left')
df.shape

(275547, 1085)

time: 5.56 s (started: 2021-09-07 12:19:53 +00:00)


In [100]:
df = df.merge(features2, on='user_id', how='left')
df.shape

(275547, 1104)

time: 4.78 s (started: 2021-09-07 12:19:58 +00:00)


In [101]:
df = df.merge(features3, on='user_id', how='left')
df.shape

(275547, 1105)

time: 1.6 s (started: 2021-09-07 12:20:03 +00:00)


In [102]:
df.to_parquet('../data/processed/features.parquet')

time: 18.6 s (started: 2021-09-07 12:20:05 +00:00)


### 4.1 Splitting into train and test features

In [ ]:
train_features = df[df.train_flag == 1]
train_features.shape

In [ ]:
train_features.to_parquet('../data/processed/train_features.parquet')

In [ ]:
test_features = df[df.train_flag == 0]
test_features.shape

In [ ]:
test_features.to_parquet('../data/processed/test_features.parquet')